# Code For Extracting Multiple Images From the Google Drive and Display Pose of that images.


In [1]:
import cv2 as cv
import glob as gb
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/Google_Drive')

Mounted at /content/Google_Drive


In [3]:
path=r'/content/Google_Drive/MyDrive/All/College /Project/Human Pose Estimation Using Open CV/Graph'
net= cv.dnn.readNetFromTensorflow(path)
path =gb.glob('/content/Google_Drive/MyDrive/All/College /Project/Human Pose Estimation Using Open CV/Images/Input/*.jpg')

NameError: ignored

In [4]:
inWidth = 452
inHeight = 452
thr = 0.2

In [5]:
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

In [6]:
def pose_estimation(frame):
    frameWidth=frame.shape[1]
    frameHeight=frame.shape[0]
    net.setInput(cv.dnn.blobFromImage(frame,1.0,(inWidth,inHeight),(127.5,127.5,127.5),swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  #output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])
    
    points = [] # Key Points
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    return frame

In [2]:
count=1
font1 = {'family':'serif','color':'#238975','size':30}
for file in path:
    img = cv.imread(file)
    plt.figure(figsize = (12,12))
    str="Input Image (Part 1)- {}".format(count)
    plt.title(str, fontdict = font1,pad='14.0')
    plt.imshow(img)
    plt.figure(figsize = (12,12))
    str="Input Image (Part 2)- {}".format(count)
    plt.title(str, fontdict = font1,pad='14.0')
    plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB))
    plt.figure(figsize = (12,12))
    str="Output Image- {}".format(count) 
    plt.title(str, fontdict = font1,pad='14.0')
    est=pose_estimation(img)
    plt.imshow(cv.cvtColor(est,cv.COLOR_BGR2RGB))
    take=file.replace('Input','Output')
    plt.savefig(take)
    count+=1

NameError: ignored

In [1]:
!pwd

/content
